# Cancer Detection
## Class Activation Maps

## Load Package

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import pickle
import cv2
from tqdm import tqdm 
import matplotlib as mpl

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import zipfile 

## Load Dataframe

In [ ]:
train = pd.read_csv("../input/histopathologic-cancer-detection/train_labels.csv", dtype=str)
print(train.shape)

In [ ]:
train['id'] = train['id'].astype(str) + '.tif'


In [ ]:
train.head(10)

## View Sample of Images

In [ ]:
# Sample 16 images from the training set and display these along with their labels.

plt.figure(figsize=(10,10)) # specifying the overall grid size

for i in range(16):
    plt.subplot(4,4,i+1)    # the number of images in the grid is 6*6 (16)
    img = mpimg.imread(f'../input/histopathologic-cancer-detection/train/{train["id"][i]}')
    plt.imshow(img)
    plt.text(0, -5, f'Label {train["label"][i]}')
    plt.axis('off')
    
plt.tight_layout()
plt.show()

## Load Model

In [ ]:
cnn = load_model('../input/cancer-detection-models/cancer_model.h5')
cnn.summary()

## Heatmap Functions

In [ ]:
def create_grad_model(model):
    for layer in reversed(model.layers):
        if len(layer.output_shape) == 4:
            last_conv_layer = layer.name
            break

    grad_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer).output, model.output])
    
    return grad_model 

def compute_heatmap(image, class_ix, grad_model):

    with tf.GradientTape() as tape:
        inputs = tf.cast(image, tf.float32)
        (conv_outputs, predictions) = grad_model(inputs)
        loss = predictions[:, class_ix]
    grads = tape.gradient(loss, conv_outputs)

    cast_conv_outputs = tf.cast(conv_outputs > 0, "float32")
    cast_grads = tf.cast(grads > 0, "float32")
    guided_grads = cast_conv_outputs * cast_grads * grads

    conv_outputs = conv_outputs[0]
    guided_grads = guided_grads[0]

    weights = tf.reduce_mean(guided_grads, axis=(0, 1))

    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs), axis=-1)

    (w, h) = (image.shape[2], image.shape[1])
    heatmap = cv2.resize(cam.numpy(), (w, h))
        
    return heatmap

## Heatmaps

In [ ]:
# Create Gradient Model
gm = create_grad_model(cnn)

# Select Image and Create Heatmap
# f'../input/histopathologic-cancer-detection/train/{train["id"][i]}.tif'
filename = train["id"][0]
# img = mpimg.imread(f'train/{filename}')
img = mpimg.imread(f'../input/histopathologic-cancer-detection/train/{train["id"][0]}')   
img = cv2.resize(img, (32, 32))
tensor = img.reshape(1,32,32,3) / 255

# img = mpimg.imread(f'../input/histopathologic-cancer-detection/train/{filename}')
# tensor = img.reshape((1,) + img.shape) / 255
heatmap = compute_heatmap(tensor, 1, gm)

plt.figure(figsize=[9,3])

# Display Image
plt.subplot(1,3,1)
plt.imshow(img)
plt.axis('off')

# Display Heatmap
plt.subplot(1,3,2)
plt.imshow(heatmap, cmap='coolwarm')
plt.axis('off')

# Display Image and Heatmap Together
plt.subplot(1,3,3)
plt.imshow(img, alpha=0.8, cmap='binary_r')
plt.imshow(heatmap, alpha=0.6, cmap='coolwarm')
plt.axis('off')
    
plt.show()

## Multiple Heatmaps

In [ ]:
def get_heatmap_dist(df, class_ix, gm):

    values = None
    for i, row in tqdm(df.iterrows()):
#         f'../input/histopathologic-cancer-detection/train/{train["id"][0]}'
#         img = mpimg.imread(f'train/{row.id}')    
        img = mpimg.imread(f'../input/histopathologic-cancer-detection/train/{train["id"][i]}')   
        img = cv2.resize(img, (32, 32))
        tensor = img.reshape(1,32,32,3) / 255
        hm = compute_heatmap(tensor, class_ix, gm)

        if values is None:
            values = hm.flatten()
        else:
            values = np.hstack([values, hm.flatten()])

    return values

In [ ]:
values = get_heatmap_dist(train.sample(1000, random_state=1), 1, gm)

In [ ]:
# setting a high end and low end of the color
low = np.quantile(values, 0.10)
high = np.quantile(values, 0.96)

norm = mpl.colors.Normalize(vmin=low, vmax=high)

print(low)
print(high)

In [ ]:
# Select which images to display
indices = range(12)

for i in indices:  
    row = train.iloc[i,:]
    img = mpimg.imread(f'../input/histopathologic-cancer-detection/train/{train["id"][i]}')    
    label = row.label
    img = cv2.resize(img, (32, 32))
    tensor = img.reshape(1,32,32,3) / 255
    heatmap = compute_heatmap(tensor, 1, gm)

    if(label == '1'):
        print('Cancer Present')
    else:
        print('No Cancer')
    
    plt.figure(figsize=[9,3])

    plt.subplot(1,3,1)
    plt.imshow(img)
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(heatmap, cmap='coolwarm', norm=norm)
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(img, alpha=0.6, cmap='binary_r')
    plt.imshow(heatmap, alpha=0.6, cmap='coolwarm', norm=norm)
    plt.axis('off')
    
    plt.show()